In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
HRD_file = pd.read_excel("DATA_ROOT_DIR/HRD_Score.xlsx")

In [3]:
HRD_file_BRCA = HRD_file[HRD_file['cancer type']=='BRCA']
label = ['patient ID', 'HRD_Status']
BRCA_data=pd.DataFrame()
for i in label:
    BRCA_data[str(i)] = HRD_file_BRCA [i] 
print(BRCA_data)

           patient ID  HRD_Status
476   TCGA-3C-AAAU-01         0.0
477   TCGA-3C-AALI-01         0.0
478   TCGA-3C-AALJ-01         0.0
479   TCGA-3C-AALK-01         0.0
480   TCGA-4H-AAAK-01         0.0
...               ...         ...
1452  TCGA-WT-AB44-01         0.0
1453  TCGA-XX-A899-01         0.0
1454  TCGA-XX-A89A-01         0.0
1455  TCGA-Z7-A8R5-01         0.0
1456  TCGA-Z7-A8R6-01         0.0

[981 rows x 2 columns]


In [4]:
# 单站点数据生成
h5_lists = os.listdir("DATA_ROOT_DIR/classification_features_dir/h5_files")
slide_lists = []

slide_data = pd.DataFrame()
for file in h5_lists:
    basename, _ = os.path.splitext(file)
    h5_patient = basename[len('slide_'):len('slide_') + 15]
    if h5_patient in BRCA_data['patient ID'].values:
        BRCA_index = BRCA_data[BRCA_data['patient ID']==h5_patient].index[0]
        new_row = pd.DataFrame([{
            "case_id": h5_patient, 
            "slide_id": basename, 
            "censorship": "class_" + str(int(BRCA_data.at[BRCA_index, 'HRD_Status'])), 
            "institute": "site_0"
        }])
        slide_data = pd.concat([slide_data, new_row], ignore_index=True)
        # 
        slide_data.to_csv("./dataset_csv/BRCA_nofl_data.csv", index=False)

In [5]:
# 多站点数据生成
site_nums = 2
h5_lists = os.listdir("DATA_ROOT_DIR/classification_features_dir/h5_files")
slide_lists = []

slide_data = pd.DataFrame()
for file in h5_lists:
    basename, _ = os.path.splitext(file)
    h5_patient = basename[len('slide_'):len('slide_') + 15]
    if h5_patient in BRCA_data['patient ID'].values:
        BRCA_index = BRCA_data[BRCA_data['patient ID']==h5_patient].index[0]
        new_row = pd.DataFrame([{
            "case_id": h5_patient, 
            "slide_id": basename, 
            "censorship": "class_" + str(int(BRCA_data.at[BRCA_index, 'HRD_Status'])), 
            "institute": ""
        }])
        slide_data = pd.concat([slide_data, new_row], ignore_index=True)
# 将slide_data按类别分组
grouped = slide_data.groupby('censorship')
# 初始化一个空的列表来存储每个站点的数据
site_dataframes = [[] for _ in range(site_nums)]
# 按类别将数据分配到各个站点
for _, group in grouped:
    # 将每个类别的数据按站点数量分块
    chunks = np.array_split(group, site_nums)
    for idx, chunk in enumerate(chunks):
        site_dataframes[idx].append(chunk)
        chunk['institute'] = f"site_{idx}"
        slide_data = pd.concat([slide_data, chunk], ignore_index=True)
# print(slide_data)
# 输出当前site_data中site0的个数以及site1的个数
site0_count = len(slide_data[slide_data['institute'] == 'site_0'])
site1_count = slide_data[slide_data['institute'] == 'site_1'].shape[0]

# 输出site0中class_0的个数
site0_class0_count = slide_data[(slide_data['institute'] == 'site_0') & (slide_data['censorship'] == 'class_0')].shape[0]
site0_class1_count = slide_data[(slide_data['institute'] == 'site_1') & (slide_data['censorship'] == 'class_1')].shape[0]
# 输出site1中class_0的个数
site1_class0_count = slide_data[(slide_data['institute'] == 'site_1') & (slide_data['censorship'] == 'class_0')].shape[0]
site1_class1_count = slide_data[(slide_data['institute'] == 'site_1') & (slide_data['censorship'] == 'class_1')].shape[0]
print(f"site_0 count: {site0_count}")
print(f"site_1 count: {site1_count}")
print(f"site_0 class_0 count: {site0_class0_count}")
print(f"site_0 class_1 count: {site0_class1_count}")
print(f"site_1 class_0 count: {site1_class0_count}")
print(f"site_1 class_1 count: {site1_class1_count}")

slide_data.to_csv("./dataset_csv/BRCA_fl_data.csv", index=False)

site_0 count: 507
site_1 count: 505
site_0 class_0 count: 413
site_0 class_1 count: 93
site_1 class_0 count: 412
site_1 class_1 count: 93


/home/lzhao/anaconda/envs/clam_latest/lib/python3.10/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [33]:
# 划分数据集
from sklearn.model_selection import train_test_split

def split_train_test_valid(input_path, file, site_nums):
    res_df = pd.DataFrame() 
    # read file
    df = pd.read_csv(input_path + file)
    for site_id in range(site_nums):
        train, valid, test =[],[],[]
        df['institute'] = df['institute'].str.slice(-1)
        df_cur = df[df['institute'] == str(site_id)]['slide_id']
        df_cur = df_cur.reset_index(drop=True)
        # define the ratios 8:1:1
        # Split into train and temp (validation + test)
        train, temp = train_test_split(df_cur, test_size=0.2, random_state=42)
        train= train.reset_index(drop=True)
        # Split temp into validation and test
        valid, test = train_test_split(temp, test_size=0.5, random_state=42)
        valid= valid.reset_index(drop=True)
        test= test.reset_index(drop=True)

        tmp ={"train":train, "val":valid, "test" : test}
        df_col = pd.DataFrame(tmp)
        print(df_col.shape)
        # print(df_col)
        # df_col.to_csv("00.csv")
        res_df = pd.concat([res_df, df_col], ignore_index=True)  

    # res_df.to_csv("00.csv") 
    # print(res_df)
    return res_df
input_path = "./dataset_csv/"
file = "BRCA_fl_data.csv"
site_nums = len(pd.read_csv(input_path + file)['institute'].dropna().unique())
print("site_nums:", site_nums)
# file = "BRCA_nofl_data.csv"
split_train_test_valid(input_path, file, site_nums)
    

In [34]:
input_path = "./dataset_csv/"
# file = "classification_hrd_dataset_fl.csv"
# file = "classification_hrd_dataset.csv"
# file = "BRCA_fl_data.csv"
file = "BRCA_nofl_data.csv"
k = 5
site_nums = len(pd.read_csv(input_path + file)['institute'].dropna().unique())
print("site_nums:", site_nums)
for kindex in range(k):
    res = split_train_test_valid(input_path, file, site_nums)
    if not os.path.exists("./splits/fl_classification"):
        os.makedir("./splits/fl_classification")
    res.to_csv("./splits/fl_classification/splits_{}.csv".format(kindex))

site_nums: 1
(809, 3)
(809, 3)
(809, 3)
(809, 3)
(809, 3)


In [7]:
input_path = "./dataset_csv/"
# file = "classification_hrd_dataset_fl.csv"
# file = "classification_hrd_dataset.csv"
# file = "BRCA_fl_data.csv"
file = "BRCA_nofl_data.csv"
k = 5
site_nums = len(pd.read_csv(input_path + file)['institute'].dropna().unique())
print("site_nums:", site_nums)
for kindex in range(k):
    res = split_train_test_valid(input_path, file, site_nums)
    for index, row in res.iterrows():
        site_name = row['site']
        site_data = row['data']
        # site_data.to_csv("00.csv")
        # print(site_data)
        # if not os.path.exists("./splits/fl_classification".format(site_name)):
        #     os.makedirs("./splits/fl_classification/site_{}".format(site_name))
        if not os.path.exists("./splits/nofl_classification".format(site_name)):
            os.makedir("./splits/nofl_classification".format(site_name))
        print(kindex, "次划分:", site_name, ":train", len(site_data["train"].dropna()), ",val", len(site_data["val"].dropna()), ",test", len(site_data["test"].dropna()))
        site_data.to_csv("./splits/nofl_classification/splits_{}.csv".format(kindex))

site_nums: 1


KeyError: 'site'